In [ ]:
#📒 Notebook 03 — Price Elasticity Model (Paso 1: Filtros de calidad)


In [52]:
#0) Setup + carga del dataset

import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:,.4f}'.format)

# Usamos Parquet:

eda_df = pd.read_parquet("eda_sales_prices.parquet")

# Asegura tipos
eda_df['date'] = pd.to_datetime(eda_df['date'])
eda_df['sell_price'] = pd.to_numeric(eda_df['sell_price'], errors='coerce')
eda_df['units_sold'] = pd.to_numeric(eda_df['units_sold'], errors='coerce')

In [ ]:
#Sanity check

In [53]:
print("eda_df shape:", eda_df.shape)
print("Nulls:\n", eda_df[['sell_price','units_sold','country','item_id','store_id','date']].isna().sum())


eda_df shape: (1999645, 13)
Nulls:
 sell_price    0
units_sold    0
country       0
item_id       0
store_id      0
date          0
dtype: int64


In [ ]:
#1) Define la “unidad de análisis” para elasticidad

#Para pricing, lo más defensible es estimar elasticidad por:

#✅ item_id + store_id + country
#(“este producto en este mercado/tienda”)

In [54]:
#Creamos el key:

eda_df['series_key'] = (
    eda_df['item_id'].astype(str) + "_" +
    eda_df['store_id'].astype(str) + "_" +
    eda_df['country'].astype(str)
)


In [ ]:
#2) Filtro base: datos útiles para log-log

In [55]:
df = eda_df.dropna(subset=['sell_price','units_sold']).copy()
df = df[(df['sell_price'] > 0) & (df['units_sold'] > 0)].copy()

print("After base filter shape:", df.shape)


After base filter shape: (1999645, 14)


In [ ]:
#3) Filtros de calidad por serie

#Vamos a filtrar series que:

#tengan suficientes observaciones (ej. ≥ 60)

#tengan variación de precio (≥ 3 precios distintos)

#no sean “precio casi fijo” (CV mínimo)

In [56]:
#Parametros iniciales (a manipular)

MIN_OBS = 60
MIN_UNIQUE_PRICES = 3
MIN_PRICE_CV = 0.02   # 2% de variación relativa

In [57]:
#Calcula métricas por serie

series_stats = df.groupby('series_key').agg(
    obs=('units_sold','size'),
    unique_prices=('sell_price', 'nunique'),
    price_mean=('sell_price','mean'),
    price_std=('sell_price','std')
).reset_index()

series_stats['price_cv'] = series_stats['price_std'] / series_stats['price_mean']

series_stats.describe()


,obs,unique_prices,price_mean,price_std,price_cv
count,"12,371.0000","12,371.0000","12,371.0000","12,366.0000","12,366.0000"
mean,161.6397,1.3376,4.1451,0.0318,0.0094
std,91.8261,0.6022,3.9171,0.0952,0.0216
min,1.0000,1.0000,0.1997,0.0000,0.0000
25%,86.0000,1.0000,1.9800,0.0000,0.0000
50%,152.0000,1.0000,2.8988,0.0000,0.0000
75%,231.0000,2.0000,4.7409,0.0174,0.0061
max,365.0000,7.0000,29.9600,3.7552,0.4664


In [58]:
#Aplicamos filtros

good_series = series_stats[
    (series_stats['obs'] >= MIN_OBS) &
    (series_stats['unique_prices'] >= MIN_UNIQUE_PRICES) &
    (series_stats['price_cv'] >= MIN_PRICE_CV)
]['series_key']

df_filt = df[df['series_key'].isin(good_series)].copy()

print("Series before:", series_stats.shape[0])
print("Series after:", df_filt['series_key'].nunique())
print("Rows after:", df_filt.shape[0])
print("Retention % (rows):", round(100*df_filt.shape[0]/df.shape[0], 2))


Series before: 12371
Series after: 516
Rows after: 91536
Retention % (rows): 4.58


In [ ]:
#4 Winsorizar outliers extremos de precio

In [59]:
p_lo, p_hi = df_filt['sell_price'].quantile([0.01, 0.99])
df_filt = df_filt[(df_filt['sell_price'] >= p_lo) & (df_filt['sell_price'] <= p_hi)].copy()

print("After winsor-ish filter:", df_filt.shape)


After winsor-ish filter: (91034, 14)


In [ ]:
#5 Guardamos dataset filtrado

In [60]:
df_filt.to_parquet("elasticity_input.parquet", index=False)
# o CSV si prefieres
df_filt.to_csv("elasticity_input.csv", index=False)


In [ ]:
#6️⃣ Preparación para OLS

In [61]:
import statsmodels.api as sm

In [62]:
#Creamos logs:

df_filt['log_units'] = np.log(df_filt['units_sold'])
df_filt['log_price'] = np.log(df_filt['sell_price'])

In [ ]:
#7️⃣ Modelo OLS por serie (elasticidad individual)

In [63]:
#Empezamos con una sola serie para validar.

sample_key = df_filt['series_key'].iloc[0]
sample_df = df_filt[df_filt['series_key'] == sample_key]

X = sm.add_constant(sample_df['log_price'])
y = sample_df['log_units']

model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_units   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     2.661
Date:                Mon, 26 Jan 2026   Prob (F-statistic):              0.105
Time:                        02:41:19   Log-Likelihood:                -53.520
No. Observations:                 134   AIC:                             111.0
Df Residuals:                     132   BIC:                             116.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8398      0.653     -1.286      0.201      -2.131       0.452
log_price      0.6452      0.396      1.631      0.105      -0.137       1.428
==============================================================================
Omnibus:                       23.166   Durbin-Watson:                   2.263
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               30.607
Skew:                           1.170   Prob(JB):                     2.26e-07
Kurtosis:                       3.065   Cond. No.                         46.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
#8️⃣ Función para correr elasticidad en todas las series

#Estimé elasticidades precio-demanda a nivel producto-tienda. Encontré que muchas series individuales no muestran una relación 
#estadísticamente significativa cuando se modelan únicamente con precio, lo que refleja la complejidad real del retail. 
#Al introducir controles mínimos de tendencia temporal, la estabilidad y el signo económico de las elasticidades mejora 
#sustancialmente.

In [64]:
# Prueba con otra serie aleatoria
sample_key = np.random.choice(df_filt['series_key'].unique())
sample_df = df_filt[df_filt['series_key'] == sample_key]

X = sm.add_constant(sample_df['log_price'])
y = sample_df['log_units']

sm.OLS(y, X).fit().summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_units   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     1.853
Date:                Mon, 26 Jan 2026   Prob (F-statistic):              0.177
Time:                        02:41:22   Log-Likelihood:                -63.394
No. Observations:                  90   AIC:                             130.8
Df Residuals:                      88   BIC:                             135.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1650      0.551      2.116      0.037       0.071       2.259
log_price     -0.5723      0.420     -1.361      0.177      -1.408       0.263
==============================================================================
Omnibus:                       10.239   Durbin-Watson:                   1.987
Prob(Omnibus):                  0.006   Jarque-Bera (JB):               11.107
Skew:                           0.860   Prob(JB):                      0.00387
Kurtosis:                       2.999   Cond. No.                         21.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [65]:
#Agregar control mínimo de tendencia

sample_df = sample_df.sort_values('date')
sample_df['t'] = np.arange(len(sample_df))

X = sm.add_constant(sample_df[['log_price','t']])
y = sample_df['log_units']

sm.OLS(y, X).fit().summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_units   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.058
Date:                Mon, 26 Jan 2026   Prob (F-statistic):              0.351
Time:                        02:41:26   Log-Likelihood:                -63.250
No. Observations:                  90   AIC:                             132.5
Df Residuals:                      87   BIC:                             140.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2641      0.584      2.166      0.033       0.104       2.424
log_price     -0.6114      0.429     -1.426      0.157      -1.463       0.241
t             -0.0011      0.002     -0.528      0.599      -0.005       0.003
==============================================================================
Omnibus:                       11.218   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               12.214
Skew:                           0.901   Prob(JB):                      0.00223
Kurtosis:                       3.109   Cond. No.                         710.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
#A nivel serie individual, el precio NO es el principal driver de demanda diaria para muchos productos en M5.

In [ ]:
#Agregamos a nivel semanal:

In [66]:
df_weekly = (
    df_filt
    .groupby(['item_id','store_id','country','wm_yr_wk'], observed=True)
    .agg(
        units_sold=('units_sold','sum'),
        sell_price=('sell_price','mean')
    )
    .reset_index()
)

print(df_weekly.shape)
df_weekly.head()



(22587, 6)


,item_id,store_id,country,wm_yr_wk,units_sold,sell_price
0,FOODS_3_002,CA_3,US,11513,12,2.5000
1,FOODS_3_002,CA_3,US,11517,4,1.5000
2,FOODS_3_002,CA_3,US,11518,22,2.5000
3,FOODS_3_002,CA_3,US,11519,20,2.5000
4,FOODS_3_002,CA_3,US,11520,19,2.5000


In [67]:
df_filt[['item_id','store_id','country','wm_yr_wk']].dtypes


item_id     category
store_id    category
country     category
wm_yr_wk       int64
dtype: object

In [68]:
print("Nulls weekly:\n", df_weekly[['units_sold','sell_price']].isna().sum())
print("Unique weeks:", df_weekly['wm_yr_wk'].nunique())
print("Min/Max week:", df_weekly['wm_yr_wk'].min(), df_weekly['wm_yr_wk'].max())


Nulls weekly:
 units_sold    0
sell_price    0
dtype: int64
Unique weeks: 53
Min/Max week: 11513 11613


In [69]:
#OLS Semanal log-log

import statsmodels.api as sm
import numpy as np

df_weekly = df_weekly[(df_weekly['units_sold'] > 0) & (df_weekly['sell_price'] > 0)].copy()

df_weekly['log_units'] = np.log(df_weekly['units_sold'])
df_weekly['log_price'] = np.log(df_weekly['sell_price'])


In [70]:
#Filtros de calidad (semanal)

df_weekly['series_key'] = (
    df_weekly['item_id'].astype(str) + "_" +
    df_weekly['store_id'].astype(str) + "_" +
    df_weekly['country'].astype(str)
)

MIN_WEEKS = 26               # al menos medio año de semanas
MIN_UNIQUE_PRICES_W = 3      # variación real de precio en semanas

wk_stats = df_weekly.groupby('series_key').agg(
    obs=('log_units','size'),
    unique_prices=('sell_price','nunique')
).reset_index()

good_keys = wk_stats[
    (wk_stats['obs'] >= MIN_WEEKS) &
    (wk_stats['unique_prices'] >= MIN_UNIQUE_PRICES_W)
]['series_key']

df_w = df_weekly[df_weekly['series_key'].isin(good_keys)].copy()

print("Weekly series before:", wk_stats.shape[0])
print("Weekly series after:", df_w['series_key'].nunique())
print("Weekly rows after:", df_w.shape[0])


Weekly series before: 513
Weekly series after: 499
Weekly rows after: 22167


In [71]:
#Probar OLS más amplia

top_key = df_w.groupby('series_key').size().sort_values(ascending=False).index[0]
sample_w = df_w[df_w['series_key'] == top_key].copy()

X = sm.add_constant(sample_w['log_price'])
y = sample_w['log_units']
res = sm.OLS(y, X).fit()

print("series_key:", top_key)
print("elasticity:", res.params['log_price'])
print("p_value:", res.pvalues['log_price'])
print("r2:", res.rsquared)

res.summary()


series_key: FOODS_3_396_CA_3_US
elasticity: -1.7191495618586812
p_value: 0.19861813092462738
r2: 0.03219090769506783


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_units   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     1.696
Date:                Mon, 26 Jan 2026   Prob (F-statistic):              0.199
Time:                        02:41:54   Log-Likelihood:                -30.944
No. Observations:                  53   AIC:                             65.89
Df Residuals:                      51   BIC:                             69.83
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.1536      3.125      1.969      0.054      -0.121      12.428
log_price     -1.7191      1.320     -1.302      0.199      -4.369       0.931
==============================================================================
Omnibus:                       46.853   Durbin-Watson:                   1.411
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              191.259
Skew:                          -2.399   Prob(JB):                     2.94e-42
Kurtosis:                      10.975   Cond. No.                         144.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [48]:
##OLS semanal con controles mínimos

#Vamos a meter un control simple y defendible:

#tendencia temporal (ordinal de semana)
#efecto fijo de semana (seasonality ligera) opcional

#Primero: tendencia temporal

In [72]:
#Creamos variable t_week

df_w = df_w.sort_values(['series_key','wm_yr_wk']).copy()
df_w['t_week'] = df_w.groupby('series_key').cumcount()


In [73]:
#Correr OLS con t_week

top_key = df_w.groupby('series_key').size().sort_values(ascending=False).index[0]
sample_w = df_w[df_w['series_key'] == top_key].copy()

X = sm.add_constant(sample_w[['log_price','t_week']])
y = sample_w['log_units']
res2 = sm.OLS(y, X).fit()

print("series_key:", top_key)
print("elasticity:", res2.params['log_price'])
print("p_value:", res2.pvalues['log_price'])
print("r2:", res2.rsquared)

res2.summary()


series_key: FOODS_3_396_CA_3_US
elasticity: -2.1055141320492203
p_value: 0.45458226725690476
r2: 0.032669972653667334


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_units   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.8443
Date:                Mon, 26 Jan 2026   Prob (F-statistic):              0.436
Time:                        02:42:08   Log-Likelihood:                -30.930
No. Observations:                  53   AIC:                             67.86
Df Residuals:                      50   BIC:                             73.77
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.0339      6.423      1.095      0.279      -5.867      19.935
log_price     -2.1055      2.794     -0.754      0.455      -7.717       3.506
t_week         0.0013      0.008      0.157      0.876      -0.016       0.018
==============================================================================
Omnibus:                       47.109   Durbin-Watson:                   1.414
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              193.477
Skew:                          -2.412   Prob(JB):                     9.70e-43
Kurtosis:                      11.021   Cond. No.                     3.45e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.45e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [74]:
res2_rob = sm.OLS(y, X).fit(cov_type='HC3')

print("elasticity:", res2_rob.params['log_price'])
print("p_value:", res2_rob.pvalues['log_price'])
print("r2:", res2_rob.rsquared)

res2_rob.summary()


elasticity: -2.1055141320492203
p_value: 0.3130387249669234
r2: 0.032669972653667334


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_units   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.7676
Date:                Mon, 26 Jan 2026   Prob (F-statistic):              0.470
Time:                        02:42:12   Log-Likelihood:                -30.930
No. Observations:                  53   AIC:                             67.86
Df Residuals:                      50   BIC:                             73.77
Df Model:                           2                                         
Covariance Type:                  HC3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.0339      4.798      1.466      0.143      -2.370      16.438
log_price     -2.1055      2.087     -1.009      0.313      -6.196       1.985
t_week         0.0013      0.008      0.159      0.874      -0.015       0.018
==============================================================================
Omnibus:                       47.109   Durbin-Watson:                   1.414
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              193.477
Skew:                          -2.412   Prob(JB):                     9.70e-43
Kurtosis:                      11.021   Cond. No.                     3.45e+03
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
[2] The condition number is large, 3.45e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
#1️⃣ Lectura de OLS (L45 y L46)
#Elasticidad

#Elasticidad ≈ −2.10

#Signo económico: ✅ correcto (muy elástica)

#Significancia

#p-value OLS: 0.45

#p-value robusto (HC3): 0.31

#❌ No significativa.

#R² ~0.033

#Muy bajo → precio explica muy poco de la variación semanal

#Tendencia (t_week)

#Coeficiente ~0

#No significativa

#📌 Conclusión estadística:

#Incluso con agregación semanal y tendencia, el precio no es un driver estadísticamente fuerte de la demanda para esta serie individual.#

In [ ]:
#🅰️ Elasticidad POOLED por dept_id + country

#👉 Alta interpretabilidad + alta estabilidad

#Modelo:

#log(Qw​)=β0​+β1​log(Pw​)+β2​t+ϵ

#🅱️ Elasticidad REGULARIZADA (Ridge / Lasso)

#👉 Reduce ruido entre series individuales
#👉 Ideal para mostrar madurez técnica

In [ ]:
#4.1 — Modelo POOLED

In [76]:
item_dept_map = (
    df_filt[['item_id', 'dept_id']]
    .drop_duplicates()
)
print(item_dept_map.shape)
item_dept_map.head()


(184, 2)


,item_id,dept_id
126,HOBBIES_1_310,HOBBIES_1
148,HOBBIES_1_358,HOBBIES_1
185,FOODS_3_012,FOODS_3
211,FOODS_3_057,FOODS_3
226,FOODS_3_081,FOODS_3


In [77]:
check = item_dept_map.groupby('item_id')['dept_id'].nunique().max()
print("Max de dept por item:", check)   # debería ser 1


Max de dept por item: 1


/tmp/ipykernel_436/2101799735.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  check = item_dept_map.groupby('item_id')['dept_id'].nunique().max()


In [78]:
#Merge al weekly
df_weekly = df_weekly.merge(item_dept_map, on='item_id', how='left')
print(df_weekly['dept_id'].isna().sum())
df_weekly.head()


0


,item_id,store_id,country,wm_yr_wk,units_sold,sell_price,log_units,log_price,series_key,dept_id
0,FOODS_3_002,CA_3,US,11513,12,2.5000,2.4849,0.9163,FOODS_3_002_CA_3_US,FOODS_3
1,FOODS_3_002,CA_3,US,11517,4,1.5000,1.3863,0.4055,FOODS_3_002_CA_3_US,FOODS_3
2,FOODS_3_002,CA_3,US,11518,22,2.5000,3.0910,0.9163,FOODS_3_002_CA_3_US,FOODS_3
3,FOODS_3_002,CA_3,US,11519,20,2.5000,2.9957,0.9163,FOODS_3_002_CA_3_US,FOODS_3
4,FOODS_3_002,CA_3,US,11520,19,2.5000,2.9444,0.9163,FOODS_3_002_CA_3_US,FOODS_3


In [80]:

df_weekly['series_key'] = (
    df_weekly['item_id'].astype(str) + "_" +
    df_weekly['store_id'].astype(str) + "_" +
    df_weekly['country'].astype(str)
)

df_weekly['log_units'] = np.log(df_weekly['units_sold'])
df_weekly['log_price'] = np.log(df_weekly['sell_price'])

wk_stats = df_weekly.groupby('series_key').agg(
    obs=('log_units','size'),
    unique_prices=('sell_price','nunique')
).reset_index()

good_keys = wk_stats[
    (wk_stats['obs'] >= MIN_WEEKS) &
    (wk_stats['unique_prices'] >= MIN_UNIQUE_PRICES_W)
]['series_key']

df_w = df_weekly[df_weekly['series_key'].isin(good_keys)].copy()
print(df_w.columns)


Index(['item_id', 'store_id', 'country', 'wm_yr_wk', 'units_sold',
       'sell_price', 'log_units', 'log_price', 'series_key', 'dept_id'],
      dtype='object')


In [81]:
#Dataset pooled

df_pool = (
    df_w
    .groupby(['dept_id','country','wm_yr_wk'], observed=True)
    .agg(
        units_sold=('units_sold','sum'),
        sell_price=('sell_price','mean')
    )
    .reset_index()
)

df_pool = df_pool[(df_pool['units_sold'] > 0) & (df_pool['sell_price'] > 0)].copy()
df_pool['log_units'] = np.log(df_pool['units_sold'])
df_pool['log_price'] = np.log(df_pool['sell_price'])

df_pool = df_pool.sort_values(['dept_id','country','wm_yr_wk'])
df_pool['t'] = df_pool.groupby(['dept_id','country']).cumcount()

print(df_pool.shape)
df_pool.head()


(318, 8)


/tmp/ipykernel_436/4150629972.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_pool['t'] = df_pool.groupby(['dept_id','country']).cumcount()


,dept_id,country,wm_yr_wk,units_sold,sell_price,log_units,log_price,t
0,FOODS_3,US,11513,4272,3.2770,8.3598,1.1869,0
1,FOODS_3,US,11514,5289,3.2291,8.5734,1.1722,1
2,FOODS_3,US,11515,3998,3.1224,8.2935,1.1386,2
3,FOODS_3,US,11516,1961,3.3497,7.5812,1.2089,3
4,FOODS_3,US,11517,2225,3.3045,7.7075,1.1953,4


In [82]:
#OLS Pooled con SE reobustos

import statsmodels.api as sm

X = sm.add_constant(df_pool[['log_price', 't']])
y = df_pool['log_units']

pool_res = sm.OLS(y, X).fit(cov_type='HC3')

print("Elasticidad pooled:", pool_res.params['log_price'])
print("p_value:", pool_res.pvalues['log_price'])
print("R2:", pool_res.rsquared)

pool_res.summary()


Elasticidad pooled: -4.32499779314131
p_value: 8.859396573201024e-55
R2: 0.268225958744134


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_units   R-squared:                       0.268
Model:                            OLS   Adj. R-squared:                  0.264
Method:                 Least Squares   F-statistic:                     124.1
Date:                Mon, 26 Jan 2026   Prob (F-statistic):           1.79e-40
Time:                        02:49:58   Log-Likelihood:                -520.07
No. Observations:                 318   AIC:                             1046.
Df Residuals:                     315   BIC:                             1057.
Df Model:                           2                                         
Covariance Type:                  HC3                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         11.5332      0.382     30.192      0.000      10.785      12.282
log_price     -4.3250      0.277    -15.587      0.000      -4.869      -3.781
t              0.0061      0.005      1.320      0.187      -0.003       0.015
==============================================================================
Omnibus:                       31.952   Durbin-Watson:                   0.082
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               10.014
Skew:                          -0.037   Prob(JB):                      0.00669
Kurtosis:                       2.134   Cond. No.                         281.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [83]:
#Elasticidad por Segmento (dept+país)

results = []

for (dept, ctry), g in df_pool.groupby(['dept_id','country'], observed=True):
    X = sm.add_constant(g[['log_price','t']])
    y = g['log_units']
    res = sm.OLS(y, X).fit(cov_type='HC3')
    results.append({
        'dept_id': dept,
        'country': ctry,
        'elasticity': res.params['log_price'],
        'p_value': res.pvalues['log_price'],
        'r2': res.rsquared,
        'n_obs': int(res.nobs)
    })

seg_elast = pd.DataFrame(results).sort_values(['dept_id','country'])
seg_elast


,dept_id,country,elasticity,p_value,r2,n_obs
2,FOODS_3,ES,0.4653,0.6117,0.1635,53
1,FOODS_3,MX,-0.3655,0.6480,0.4227,53
0,FOODS_3,US,-8.5020,0.0000,0.3968,53
5,HOBBIES_1,ES,-0.1950,0.7538,0.0493,53
4,HOBBIES_1,MX,0.1982,0.5987,0.0030,53
3,HOBBIES_1,US,1.5224,0.1913,0.1665,53


In [84]:
df_pool['t'] = df_pool.groupby(['dept_id','country'], observed=True).cumcount()


In [ ]:
#1️⃣ Modelo POOLED global
#Resultados clave

#Elasticidad pooled: −4.33

#p-value: ~0 (8.9e-55) ✅

#R²: 0.268 (muy decente para retail)

#Interpretación económica

#A nivel agregado (categoría + país), un aumento de 1% en precio se asocia con una caída promedio de ~4.3% en unidades vendidas, controlando por tendencia temporal.

In [ ]:
#2️⃣ Elasticidades por segmento (dept × país)

#FOODS_3

| País   | Elasticidad | p-value   | R²    | Lectura              |
| ------ | ----------- | --------- | ----- | -------------------- |
| **US** | **−8.50**   | **0.000** | 0.397 | Altamente elástico   |
| MX     | −0.37       | 0.65      | 0.423 | Inelástico / no sig. |
| ES     | +0.47       | 0.61      | 0.164 | No interpretable     |

#Insight: FOODS en US muestra alta sensibilidad al precio; en MX/ES el precio no explica la demanda (otros factores dominan).

In [ ]:
#HOBBIES_1

| País | Elasticidad | p-value | R²    | Lectura    |
| ---- | ----------- | ------- | ----- | ---------- |
| US   | +1.52       | 0.19    | 0.167 | No estable |
| MX   | +0.20       | 0.60    | 0.003 | No señal   |
| ES   | −0.19       | 0.75    | 0.049 | No señal   |

#Insight: En HOBBIES, la relación precio-demanda no es estable con este nivel de agregación (posible efecto de promociones, lanzamientos, o demanda discrecional).

In [ ]:
#Hallazgos clave

#A nivel SKU–tienda, las estimaciones de elasticidad precio–demanda resultan inestables y mayormente no significativas, lo que refleja un alto nivel de ruido en la demanda y una variación limitada de precios a este nivel de granularidad.

#La agregación semanal mejora la estabilidad del modelo y reduce el ruido presente en los datos diarios; sin embargo, sigue siendo insuficiente para obtener estimaciones estadísticamente robustas a nivel de series individuales.

#Un modelo pooled a nivel categoría–país produce una elasticidad fuerte y estadísticamente significativa (−4.3), lo que indica una alta sensibilidad de la demanda ante cambios de precio cuando se analiza el comportamiento agregado.

#La elasticidad varía de forma considerable según la categoría y el mercado. En particular, la categoría FOODS en Estados Unidos muestra una sensibilidad al precio significativamente mayor, mientras que en otros mercados y categorías el 
#precio no emerge como el principal driver de la demanda.